# Tutorial for basics usage of the JPL Small Body Identification Tool

## Intro
The notebook demonstrates the basic usage of `JPLSBID` class which provides a wrapper and front end for the JPL [Small-Body Identification Tool](https://ssd.jpl.nasa.gov/tools/sb_ident.html). This tool  This tool allows you to search for and retrieve a list of small-bodies (asteroids and/or comets) which are likely contained in the specified fiel of view on the specified date/time. 

## Tutorial

### Import needed things


In [ ]:
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from solsys_code.views import JPLSBId

### Declare a time and point of interest 

This particular example is the (rough) start of commissioning observations on the Extended Chandra Deep Field South (E-CDF-S field)


In [ ]:
obs_time = Time('2024-11-11 09:00:00', scale='utc')
ecdfs = SkyCoord(53.13, -28.10, unit=u.deg, frame='icrs')

### Declare a JPLSBId object and initialize with the ComCam field of view (converted to a half-width)


In [ ]:
# Declare a JPLSBId object and initialize with the ComCam field of view (converted to a half-width)
rubin_comcam = JPLSBId(fov_ra_hwidth=40 * u.arcmin / 2.0, fov_dec_hwidth=40 * u.arcmin / 2.0)
print(f'MPC Code: {rubin_comcam.mpc_code} FOV= {rubin_comcam.fov_ra_hwidth:.3f} x {rubin_comcam.fov_dec_hwidth:.3f}')

### Query the JPL Small Body ID tool

This can take 30-60s. The defaults (`raw_response`=True, `verbose`=True) will show details of the query and return the raw (JSON) response


In [ ]:
results = rubin_comcam.query_center(obs_time, ecdfs)
print(results.keys())

### Looking at the summary

The basic summary of the query is in the dictionary of results under the `summary` key (the RA and Dec are '-'-separated and minus signs in the Dec are convered to `M`)


In [ ]:
for key, value in results['summary'].items():
    print(f'{key:<14}: {value}')

### Examining results
The per-object data is a list of fields (also a list) under the `data_first_pass` key with the fields ordered as described in `fields_first`

In [ ]:
print('\n'.join(results['fields_first']))
print()
print(results['data_first_pass'][0])

### Turning results into a Table
If instead, we set `raw_response` to False, the list of objects will be turned into a `QTable` with the position as a `SkyCoord` and `Quantity` columns

In [ ]:
table = rubin_comcam.query_center(obs_time, ecdfs, raw_response=False)
print(table)
print(table.colnames)